In [1]:
%cd ../
import numpy as np
import pandas as pd
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

/home/harshapk/Projects/CSS_Project


In [2]:
import pickle
with open("./data/data_processed/lies_data_cleaned.pkl", "rb") as f:
    dataset = pickle.load(f)

In [3]:
dataset = dataset[dataset["lang"] == "en"]

In [4]:
dataset = dataset[["tweet_text", "retweets", "likes", "user_description", "user_following",
"user_followers", "user_verified", "user_total_tweets", "user_total_likes", "label"]]

In [5]:
dataset["label"].unique()

array(['na', 'neg', 'pos'], dtype=object)

In [6]:
from css_proj.models.bertembed import get_sentence_embed
import transformers as ppb
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from css_proj.models.clean_tweet import process_text
all_tw_text = dataset["tweet_text"].tolist()
all_tw_text_embed = [get_sentence_embed([process_text(text)], model, tokenizer).numpy() 
                     for text in tqdm(all_tw_text)]

100%|█████████████████████████████████████████████████████████████████████████████| 2891/2891 [03:35<00:00, 13.42it/s]


In [9]:
all_tw_text_embed = np.array(all_tw_text_embed)
all_tw_text_embed = all_tw_text_embed.squeeze()

In [10]:
all_tw_desc = dataset["user_description"].tolist()
all_tw_desc_embed = [get_sentence_embed([process_text(text)], model, tokenizer).numpy() for text in tqdm(all_tw_desc)]
all_tw_desc_embed = np.array(all_tw_desc_embed).squeeze()

100%|█████████████████████████████████████████████████████████████████████████████| 2891/2891 [03:39<00:00, 13.14it/s]


In [11]:
with open("./data/data_processed/covid_lies_embed.pkl", "wb") as f:
    pickle.dump([all_tw_text_embed, all_tw_desc_embed], f)

In [6]:
with open("./data/data_processed/covid_lies_embed.pkl", "rb") as f:
    all_tw_text_embed, all_tw_desc_embed = pickle.load(f)

In [7]:
dataset["num_label"] = dataset["label"].apply(lambda x: 0 if x == "na" else 1)

In [8]:
from empath import Empath
lexicon = Empath()
lexicon.create_category("medical", ["doctor", "physician", "hospital", "health", "disease"])

["doctors", "doctor", "treatment", "patient", "illness", "hospital", "health", "surgery", "condition", "medication", "patients", "disease", "treatments", "sickness", "labour", "labor", "chemo", "operation", "recovery", "surgeon", "Doctors", "psychiatrist", "medicine", "clinic", "pregnancy", "therapy", "cancer", "intensive_care", "nurse", "symptoms", "ward", "diagnosis", "therapist", "trauma", "midwife", "surgeons", "physician", "blood_tests", "injuries", "critical_condition", "infection", "psychologist", "procedure", "chemotherapy", "tumor", "pneumonia", "local_hospital", "healer", "emergency_room", "injury", "injection", "coma", "test_results", "donor", "serious_injuries", "medically", "prescribed", "emergency_surgery", "autopsy", "nurses", "side_effects", "Alzheimer", "medications", "healers", "cure", "minor_injuries", "flu", "meds", "discharged", "suffering", "scans", "pack_doctor", "tests", "cured", "complications", "life_support", "blood_transfusion", "ICU", "blood_test", "head_in

In [9]:
def get_words_from_txt(path):
    return pd.read_table(path, header=None).to_numpy().squeeze()
positive_words = get_words_from_txt("data/data_raw/positive-words.txt")
negative_words = get_words_from_txt("data/data_raw/negative-words.txt")
lexicon.create_category("positive_sent", list(positive_words))
lexicon.create_category("negative_sent", list(negative_words))

["remarkable", "admirable", "questionable", "exceptional", "astounding", "certainly", "yet", "Though", "desirable", "extraordinary", "wise", "although", "practical", "simplicity", "gifted", "pleasing", "humble", "adequate", "honorable", "outstanding", "respectable", "rarity", "bearing", "quality", "enlightened", "Yet", "potential", "agreeable", "certain", "appeal", "sensible", "lacking", "Yet", "appreciative", "endowed", "satisfactory", "valued", "truly", "devoted", "confident", "refined", "perceived", "formidable", "own_right", "successful", "knowledgeable", "astonishing", "intelligence", "Although", "ambition", "outrageous", "powerful", "thorough", "appealing", "promising", "costly", "ideal", "worthy", "wicked", "wondrous", "undoubtedly", "noble", "boast", "however", "enchanting", "superficial", "though", "miraculous", "spiritual", "potential", "primitive", "grateful", "though", "flawed", "superior", "splendid", "unlike", "inadequate", "fascinating", "individual", "Moreover", "reliab

In [10]:
dataset["user_medical"] = dataset["user_description"].apply(
    lambda x: lexicon.analyze(x, categories=["medical"])["medical"])
dataset["positive_sent"] = dataset["tweet_text"].apply(
    lambda x: lexicon.analyze(x, categories=["positive_sent", "negative_sent"])["positive_sent"])
dataset["negative_sent"] = dataset["tweet_text"].apply(
    lambda x: lexicon.analyze(x, categories=["positive_sent", "negative_sent"])["negative_sent"])

In [19]:
dataset[dataset['label']=='na']

,tweet_text,retweets,likes,user_description,user_following,user_followers,user_verified,user_total_tweets,user_total_likes,label,num_label
1,Getting coronavirus and then coughing on peopl...,1,0,Tankie,233,304,False,1873,33404,na,0
9,"“Most people (not elderly, no underlying healt...",0,0,gran,2038,2826,False,198290,80072,na,0
13,15-20% of people with COVID-19 may require hos...,0,0,"Wife, Mom, Grandmother - Retired Paramedic/FF",247,220,False,7913,40570,na,0
15,A sheriff's office tried to get meth off the s...,0,0,"WBAL NewsRadio 1090 and FM 101.5, the home of ...",4075,72865,True,125399,917,na,0
16,"In China’s War on the Coronavirus, a Community...",0,0,"Single, ""God Fearing Man""! The Lord, Loves Us ...",1632,138,False,56083,16498,na,0
...,...,...,...,...,...,...,...,...,...,...,...
6583,Scientists say a cutting-edge but untested new...,9,18,Australia's leading news source. Independent. ...,743,877029,True,230887,592,na,0
6586,One Spokane County resident is displaying symp...,13,9,The Spokesman-Review is the Spokane region's o...,1482,61932,False,70326,1545,na,0
6587,@ALBeforeAfter Orion is my favorite constellat...,0,5,18+ Adults ONLY!!! We are a couple in our 30's...,2484,67728,False,13331,75500,na,0
6589,@LindaJones_NYC China is dirty.\nWhen people c...,0,0,Livin' the dream....\nWe The People #MovedOnTr...,5001,2616,False,58217,74884,na,0


In [11]:
dataset_array = dataset[["retweets", "likes", "user_following",
"user_followers", "user_verified", "user_total_tweets", "user_total_likes", "user_medical",
                         "positive_sent", "negative_sent", "num_label"]]
dataset_array = np.array(dataset_array).astype(float)

In [12]:
dataset_array = np.hstack([all_tw_desc_embed, all_tw_text_embed, dataset_array])
dataset_x = dataset_array[:, :-1]
dataset_y = dataset_array[:, -1]

In [13]:
# Train test split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(dataset_x, dataset_y, test_size=0.4, random_state=42)

In [14]:
# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [15]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_x, train_y)

/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
# Evaluate model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
pred_y = logreg.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.909248055315471
[[1043   33]
 [  72    9]]
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      1076
         1.0       0.21      0.11      0.15        81

    accuracy                           0.91      1157
   macro avg       0.57      0.54      0.55      1157
weighted avg       0.88      0.91      0.90      1157

0.14634146341463417


In [17]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

In [18]:
# Evaluate model
pred_y = rfc.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9204840103716508
[[1061   15]
 [  77    4]]
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      1076
         1.0       0.21      0.05      0.08        81

    accuracy                           0.92      1157
   macro avg       0.57      0.52      0.52      1157
weighted avg       0.88      0.92      0.90      1157

0.08


In [19]:
# SVM
from sklearn.svm import SVC
svc = SVC(kernel="linear", random_state=42)
svc.fit(train_x, train_y)

SVC(kernel='linear', random_state=42)

In [20]:
# Evaluate model
pred_y = svc.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.8962834917891098
[[1020   56]
 [  64   17]]
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94      1076
         1.0       0.23      0.21      0.22        81

    accuracy                           0.90      1157
   macro avg       0.59      0.58      0.58      1157
weighted avg       0.89      0.90      0.89      1157

0.22077922077922077


In [21]:
# SVM with RBF kernel
svc_rbf = SVC(kernel="rbf", random_state=42)
svc_rbf.fit(train_x, train_y)

SVC(random_state=42)

In [22]:
# Evaluate model
pred_y = svc_rbf.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9299913569576491
[[1076    0]
 [  81    0]]
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1076
         1.0       0.00      0.00      0.00        81

    accuracy                           0.93      1157
   macro avg       0.46      0.50      0.48      1157
weighted avg       0.86      0.93      0.90      1157

0.0


/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [23]:
# Neural Network
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100, 100))
mlp.fit(train_x, train_y)

MLPClassifier(hidden_layer_sizes=(100, 100, 100), random_state=42)

In [24]:
# Evaluate model
pred_y = mlp.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9127052722558341
[[1047   29]
 [  72    9]]
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      1076
         1.0       0.24      0.11      0.15        81

    accuracy                           0.91      1157
   macro avg       0.59      0.54      0.55      1157
weighted avg       0.89      0.91      0.90      1157

0.15126050420168066
